# Exploración del Dataset: COMPETENCIAS

Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# Configuración
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 50)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("Librerías importadas correctamente")

Carga del Dataset

In [ ]:
# Definir rutas de forma robusta
import os
from pathlib import Path

# Obtener la ubicación del notebook
notebook_dir = Path.cwd()
print(f"Directorio actual: {notebook_dir}")

# Construir ruta al directorio data/raw
if 'notebooks' in str(notebook_dir):
    DATA_RAW_PATH = Path('../../data/raw')
    DICT_PATH = Path('../../data/diccionarios')
else:
    DATA_RAW_PATH = Path(r'E:\MTPE\data\raw')
    DICT_PATH = Path(r'E:\MTPE\data\diccionarios')

print(f"Ruta DATA_RAW_PATH: {DATA_RAW_PATH.resolve()}")
print(f"Directorio existe: {DATA_RAW_PATH.exists()}")

# Cargar dataset
archivo_csv = DATA_RAW_PATH / 'DataSet_COMPETENCIAS.csv'
print(f"Archivo CSV: {archivo_csv.name}")
print(f"Archivo existe: {archivo_csv.exists()}")

if archivo_csv.exists():
    print("\nCargando dataset...")
    try:
        df_competencias = pd.read_csv(archivo_csv, encoding='utf-8-sig')
    except:
        df_competencias = pd.read_csv(archivo_csv, encoding='latin1')
    print(f"\nDataset cargado exitosamente")
    print(f"Dimensiones: {df_competencias.shape[0]:,} filas x {df_competencias.shape[1]} columnas")
else:
    print(f"\nERROR: No se encuentra el archivo en: {archivo_csv.resolve()}")
    print(f"Por favor, verifica la ruta del archivo.")

Vista Previa - Primeras Filas

In [ ]:
print("Primeras 10 filas del dataset:")
display(df_competencias.head(10))

Vista Previa - Últimas Filas

In [ ]:
print("Últimas 10 filas del dataset:")
display(df_competencias.tail(10))

Vista Previa - Muestra Aleatoria

In [ ]:
print("Muestra aleatoria de 10 registros:")
display(df_competencias.sample(min(10, len(df_competencias)), random_state=42))

Información General del Dataset

In [ ]:
print("Información del Dataset:")
print("=" * 80)
df_competencias.info()

print("\nColumnas del Dataset:")
print("=" * 80)
for idx, col in enumerate(df_competencias.columns, 1):
    print(f"{idx:2d}. {col} ({df_competencias[col].dtype})")

Análisis de Valores Nulos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

print("Análisis de Valores Nulos:")
print("=" * 80)

null_data = pd.DataFrame({
    'Columna': df_competencias.columns,
    'Nulos': df_competencias.isnull().sum(),
    '% Nulos': (df_competencias.isnull().sum() / len(df_competencias) * 100).round(2)
}).sort_values(by='Nulos', ascending=False)

display(null_data)

if null_data['Nulos'].sum() > 0:
    plt.figure(figsize=(14, 8))
    null_cols = null_data[null_data['Nulos'] > 0]
    plt.barh(null_cols['Columna'], null_cols['% Nulos'])
    plt.xlabel('Porcentaje de Valores Nulos (%)')
    plt.title('Valores Nulos por Columna - COMPETENCIAS')
    plt.tight_layout()
    plt.show()
else:
    print("No hay valores nulos en el dataset")

Análisis de Duplicados

In [ ]:
print("Análisis de Duplicados:")
print("=" * 80)

duplicados_totales = df_competencias.duplicated().sum()
print(f"Registros duplicados (completos): {duplicados_totales:,} ({duplicados_totales/len(df_competencias)*100:.2f}%)")

# Buscar columna de ID
id_cols = [col for col in df_competencias.columns if 'ID' in col.upper()]
if id_cols:
    for id_col in id_cols:
        duplicados_id = df_competencias[id_col].duplicated().sum()
        print(f"Duplicados en {id_col}: {duplicados_id:,} ({duplicados_id/len(df_competencias)*100:.2f}%)")

        if duplicados_id > 0 and duplicados_id < 100:
            print(f"\nEjemplos de {id_col} duplicados:")
            ids_duplicados = df_competencias[df_competencias[id_col].duplicated(keep=False)][id_col].value_counts().head(10)
            display(ids_duplicados)

Estadísticas Descriptivas - Variables Numéricas

In [ ]:
print("Estadísticas Descriptivas - Variables Numéricas:")
print("=" * 80)

if 'df_competencias' in globals():
    display(df_competencias.describe())
else:
    print("ERROR: Primero debes ejecutar la celda 'Carga del Dataset' para cargar df_competencias")

Estadísticas Descriptivas - Variables Categóricas

In [ ]:
print("Estadísticas Descriptivas - Variables Categóricas:")
print("=" * 80)

if 'df_competencias' in globals():
    display(df_competencias.describe(include=['object']))
else:
    print("ERROR: Primero debes ejecutar la celda 'Carga del Dataset' para cargar df_competencias")

Análisis de Cardinalidad

In [ ]:
print("Análisis de Cardinalidad:")
print("=" * 80)

# Calcular valores únicos para cada columna
valores_unicos = [df_competencias[col].nunique() for col in df_competencias.columns]
tipos_dato = [str(df_competencias[col].dtype) for col in df_competencias.columns]
porcentaje_cardinalidad = [(df_competencias[col].nunique() / len(df_competencias) * 100) for col in df_competencias.columns]

cardinalidad = pd.DataFrame({
    'Columna': df_competencias.columns,
    'Valores_Únicos': valores_unicos,
    'Tipo_Dato': tipos_dato,
    '% Cardinalidad': [round(p, 2) for p in porcentaje_cardinalidad]
}).sort_values(by='Valores_Únicos', ascending=False)

display(cardinalidad)

Distribución de Variables Categóricas - Top 20

In [ ]:
print("Distribución de Variables Categóricas (Top 20 valores):")
categorical_cols = df_competencias.select_dtypes(include=['object']).columns

for col in categorical_cols:
    print(f"\n{'='*80}")
    print(f"Columna: {col}")
    print(f"{'='*80}")
    value_counts = df_competencias[col].value_counts()
    print(f"Valores únicos: {df_competencias[col].nunique():,}")
    print(f"\nTop 20 valores más frecuentes:")
    display(pd.DataFrame({
        'Valor': value_counts.head(20).index,
        'Frecuencia': value_counts.head(20).values,
        'Porcentaje': (value_counts.head(20).values / len(df_competencias) * 100).round(2)
    }))

Análisis de Nombres/Descripciones de Competencias

In [ ]:
print("Análisis de Nombres/Descripciones de Competencias:")
print("=" * 80)

# Buscar columnas con nombres o descripciones
nombre_cols = [col for col in df_competencias.columns if any(x in col.upper() for x in ['NOMBRE', 'DESCRIPCION', 'COMPETENCIA'])]

if nombre_cols:
    for nombre_col in nombre_cols:
        print(f"\n{nombre_col}:")
        nombre_counts = df_competencias[nombre_col].value_counts()

        print(f"Total de competencias únicas: {df_competencias[nombre_col].nunique():,}")
        print(f"Valores nulos: {df_competencias[nombre_col].isna().sum():,}")
        print(f"\nTop 30 competencias más frecuentes:")

        display(pd.DataFrame({
            'Competencia': [str(x)[:80] for x in nombre_counts.head(30).index],
            'Cantidad': nombre_counts.head(30).values,
            'Porcentaje': (nombre_counts.head(30).values / len(df_competencias) * 100).round(2)
        }))

        # Visualización si hay datos
        if len(nombre_counts) > 0:
            plt.figure(figsize=(14, 10))
            top_nombres = nombre_counts.head(20)
            plt.barh(range(len(top_nombres)), top_nombres.values)
            plt.yticks(range(len(top_nombres)), [str(x)[:60] + '...' if len(str(x)) > 60 else str(x) for x in top_nombres.index])
            plt.xlabel('Frecuencia')
            plt.title(f'Top 20 {nombre_col}')
            plt.tight_layout()
            plt.show()

        # Análisis de longitud
        if df_competencias[nombre_col].dtype == 'object':
            df_competencias[f'len_{nombre_col}'] = df_competencias[nombre_col].fillna('').astype(str).str.len()

            print(f"\nEstadísticas de longitud:")
            print(f"  - Longitud mínima: {df_competencias[f'len_{nombre_col}'].min()}")
            print(f"  - Longitud máxima: {df_competencias[f'len_{nombre_col}'].max()}")
            print(f"  - Longitud promedio: {df_competencias[f'len_{nombre_col}'].mean():.1f} caracteres")
            print(f"  - Longitud mediana: {df_competencias[f'len_{nombre_col}'].median():.0f} caracteres")
else:
    print("No se encontraron columnas de nombres/descripciones")

Análisis de Variables Numéricas

In [ ]:
print("Análisis Detallado de Variables Numéricas:")
print("=" * 80)

num_cols = df_competencias.select_dtypes(include=[np.number]).columns.tolist()

if num_cols:
    print(f"\nColumnas numéricas encontradas: {len(num_cols)}")
    for col in num_cols:
        print(f"\n{col}:")
        print(f"  - Mínimo: {df_competencias[col].min()}")
        print(f"  - Máximo: {df_competencias[col].max()}")
        print(f"  - Promedio: {df_competencias[col].mean():.2f}")
        print(f"  - Mediana: {df_competencias[col].median()}")
        print(f"  - Desviación estándar: {df_competencias[col].std():.2f}")
        print(f"  - Valores nulos: {df_competencias[col].isna().sum():,}")

    # Visualizaciones
    if len(num_cols) > 0:
        fig, axes = plt.subplots((len(num_cols) + 1) // 2, 2, figsize=(15, 5 * ((len(num_cols) + 1) // 2)))
        if len(num_cols) == 1:
            axes = [axes]
        else:
            axes = axes.flatten()

        for idx, col in enumerate(num_cols):
            axes[idx].hist(df_competencias[col].dropna(), bins=30, edgecolor='black')
            axes[idx].set_xlabel(col)
            axes[idx].set_ylabel('Frecuencia')
            axes[idx].set_title(f'Distribución de {col}')

        # Ocultar ejes vacíos
        for idx in range(len(num_cols), len(axes)):
            axes[idx].set_visible(False)

        plt.tight_layout()
        plt.show()
else:
    print("No se encontraron columnas numéricas")